In [ ]:
# install required packages
!pip install --quiet langchain langchain_community
!pip install --quiet langchain_google_genai
!pip install --quiet duckduckgo-search
!pip install --quiet git+https://github.com/joaomdmoura/crewai.git

In [ ]:
# imports and api key setup
import os
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_community.tools.duckduckgo_search import DuckDuckGoSearchRun
from crewai import Agent, Task, Crew, Process

os.environ["GOOGLE_API_KEY"] = "api-key"

# gemini llm initialization
llm = ChatGoogleGenerativeAI(
    model="gemini-pro",
    temperature=0.7,
    google_api_key=os.environ["GOOGLE_API_KEY"]
)

In [ ]:
# tool setup
search_tool = DuckDuckGoSearchRun()

In [ ]:
# define ai agents

# research-focused ai expert
research_agent = Agent(
    role="Tech Research Specialist",
    goal="Identify and summarize groundbreaking AI and automation developments",
    backstory="You're an analyst known for tracking disruptive AI technologies around the globe.",
    tools=[search_tool],
    verbose=True,
    allow_delegation=False,
    llm=llm
)

# content-focused tech writer
writer_agent = Agent(
    role="Tech Content Creator",
    goal="Turn research insights into a fun and accessible blog post",
    backstory="You're a blogger famous for simplifying emerging tech for curious readers.",
    tools=[],
    verbose=True,
    allow_delegation=False,
    llm=llm
)

In [ ]:
# define tasks for agents

# do the research
task_1 = Task(
    description="""
        Search and compile the most impactful AI and automation breakthroughs of 2024 so far.
        Focus on innovations, emerging tools, and societal or economic effects.
        Output must be a well-organized summary report.
    """,
    agent=research_agent
)

# write the blog post
task_2 = Task(
    description="""
        Based on the research summary, write a 4-paragraph blog article.
        Make it exciting, human, and easy to understand.
        Avoid technical jargon. Use casual language and storytelling style.
    """,
    agent=writer_agent
)

In [ ]:
# create and run the Crew
crew = Crew(
    agents=[research_agent, writer_agent],
    tasks=[task_1, task_2],
    process=Process.sequential,
    verbose=2
)

# run the pipeline
final_output = crew.kickoff()

# final result
print("\n Final Blog Post:\n")
print(final_output)